In [1]:
# Import the dependencies

import numpy as np
import pandas as pd
import re

from matplotlib import pyplot as plt
from sklearn.metrics import r2_score

from datetime import timedelta,datetime

import sys
from path import Path

# import plotly.graph_objects as go

In [2]:
# Import the user-defined ml class

class_path = Path("../classes/pmmfs_ml")
sys.path.append(class_path)
from ml import ml

In [3]:
# Load the datasets from CSV file
# Note: This will be converted to database in the final version of this file.

path_GR = "Resources/raw/OxCGRT_latest(Aug31,2020).csv"   # Government Response
path_CD = "Resources/raw/owid-covid-data(Aug31,2020).csv"   # COVID Data

In [4]:
data_GR = pd.read_csv(path_GR)    #Government Response
data_CD = pd.read_csv(path_CD)

/home/sami/anaconda3/envs/mlenv2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# Drop rows from Government response for individual regions for CountryCodes USA and GBR

data_GR = data_GR[data_GR[["RegionName"]].isnull().values].copy()

In [6]:
# Load the Stringency and Income support columns

df_GR = data_GR[['Date','CountryCode',]]
stringency_columns = [ x for x in data_GR if re.search("C\d|H1",x)]
income_support = [x for x in data_GR if re.search("E1",x)]

for x in stringency_columns + income_support:
    df_GR[x] = data_GR[x]
    
df_GR

/home/sami/anaconda3/envs/mlenv2/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,CountryCode,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,H1_Public information campaigns,H1_Flag,E1_Income support,E1_Flag
0,20200101,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
1,20200102,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
2,20200103,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
3,20200104,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
4,20200105,ABW,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58799,20200827,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58800,20200828,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58801,20200829,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58802,20200830,ZWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Count the number of nulls in each column of the government dataframe

df_GR.isnull().sum()

Date                                        0
CountryCode                                 0
C1_School closing                        1166
C1_Flag                                 18029
C2_Workplace closing                     1268
C2_Flag                                 20349
C3_Cancel public events                  1204
C3_Flag                                 17224
C4_Restrictions on gatherings            1235
C4_Flag                                 19101
C5_Close public transport                1076
C5_Flag                                 28457
C6_Stay at home requirements             1237
C6_Flag                                 23086
C7_Restrictions on internal movement     1125
C7_Flag                                 23855
C8_International travel controls         1187
H1_Public information campaigns          1667
H1_Flag                                  9947
E1_Income support                        2154
E1_Flag                                 25658
dtype: int64

In [8]:
# Show the values in each of the Stringency and Income Support columns

for x in df_GR.columns:
    if re.search("C\d|H1|E1",x):
        print(df_GR[x].value_counts())

3.0    20243
0.0    16863
2.0     5814
1.0     1054
Name: C1_School closing, dtype: int64
1.0    24094
0.0     3017
Name: C1_Flag, dtype: int64
0.0    19081
2.0    14554
3.0     6254
1.0     3983
Name: C2_Workplace closing, dtype: int64
1.0    19734
0.0     5057
Name: C2_Flag, dtype: int64
2.0    24004
0.0    16020
1.0     3912
Name: C3_Cancel public events, dtype: int64
1.0    25151
0.0     2765
Name: C3_Flag, dtype: int64
0.0    17866
4.0    12469
3.0     9677
2.0     3139
1.0      754
Name: C4_Restrictions on gatherings, dtype: int64
1.0    22667
0.0     3372
Name: C4_Flag, dtype: int64
0.0    27381
1.0     9272
2.0     7411
Name: C5_Close public transport, dtype: int64
1.0    12282
0.0     4401
Name: C5_Flag, dtype: int64
0.0    21849
2.0    10715
1.0     8905
3.0     2434
Name: C6_Stay at home requirements, dtype: int64
1.0    16532
0.0     5522
Name: C6_Flag, dtype: int64
0.0    22730
2.0    15807
1.0     5478
Name: C7_Restrictions on internal movement, dtype: int64
1.0    12912


In [9]:
# Handle missing values in the government response dataframe with zeros

df_GR = df_GR.fillna(0)

In [10]:
# Handle missing values in the COVID data dataframes

df_CD_1 = data_CD[["date","iso_code","population","population_density",'median_age',"total_cases","total_deaths"]]
df_CD_1 = df_CD_1.dropna()
df_CD_2 = data_CD[["date","iso_code","new_cases","new_deaths"]]
df_CD_2 = df_CD_2.fillna(0)

In [11]:
# Merge the COVID data dataframes

df_CD = df_CD_1.merge(df_CD_2,on=["date","iso_code"])

In [12]:
df_CD.head()

,date,iso_code,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,106766.0,584.8,41.2,2.0,0.0,2.0,0.0
1,2020-03-20,ABW,106766.0,584.8,41.2,4.0,0.0,2.0,0.0
2,2020-03-24,ABW,106766.0,584.8,41.2,12.0,0.0,8.0,0.0
3,2020-03-25,ABW,106766.0,584.8,41.2,17.0,0.0,5.0,0.0
4,2020-03-26,ABW,106766.0,584.8,41.2,19.0,0.0,2.0,0.0


In [13]:
df_CD.shape

(35174, 9)

In [14]:
# Handle nulls in the total deaths and total cases columns

codes = df_CD["iso_code"].unique()

iso_dfs = []

total_cases_col = 2
tota_deaths_col = 3

# For each country code in the COVID data
for code in codes:
    # Isolate the dataframe for that country
    iso_df = df_CD[df_CD["iso_code"] == code]
    
    # If the total cases in the first row is null, set it to 0
    if np.isnan(iso_df.iloc[0,total_cases_col]):
        iso_df.iloc[0,total_cases_col] = 0
    # IF the total deaths in the first row is null, set it to 0 
    if np.isnan(iso_df.iloc[0,tota_deaths_col]):
        iso_df.iloc[0,tota_deaths_col] = 0
    
    # Forward fill the remaining missing values
    iso_df = iso_df.fillna(method='ffill')
        
    # Add the country dataframe to the completed list
    iso_dfs.append(iso_df)

# Recreate the COVID data frame by concatenating all country dataframes
df_CD = pd.DataFrame()

for iso_df in iso_dfs:
    df_CD = df_CD.append(iso_df)
        
df_CD.isnull().sum()

date                  0
iso_code              0
population            0
population_density    0
median_age            0
total_cases           0
total_deaths          0
new_cases             0
new_deaths            0
dtype: int64

In [15]:
df_CD

,date,iso_code,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,1.067660e+05,584.800,41.2,2.0,0.0,2.0,0.0
1,2020-03-20,ABW,1.067660e+05,584.800,41.2,4.0,0.0,2.0,0.0
2,2020-03-24,ABW,1.067660e+05,584.800,41.2,12.0,0.0,8.0,0.0
3,2020-03-25,ABW,1.067660e+05,584.800,41.2,17.0,0.0,5.0,0.0
4,2020-03-26,ABW,1.067660e+05,584.800,41.2,19.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...
35169,2020-08-27,OWID_WRL,7.794799e+09,58.045,30.9,24201449.0,825936.0,276699.0,6297.0
35170,2020-08-28,OWID_WRL,7.794799e+09,58.045,30.9,24484672.0,832011.0,283223.0,6075.0
35171,2020-08-29,OWID_WRL,7.794799e+09,58.045,30.9,24761119.0,837466.0,276447.0,5455.0
35172,2020-08-30,OWID_WRL,7.794799e+09,58.045,30.9,25029850.0,843154.0,268731.0,5688.0


In [16]:
df_GR.columns

Index(['Date', 'CountryCode', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'H1_Public information campaigns',
       'H1_Flag', 'E1_Income support', 'E1_Flag'],
      dtype='object')

In [17]:
# Display count of null values in each column of the Government Response dataframe

df_GR.isnull().sum()

Date                                    0
CountryCode                             0
C1_School closing                       0
C1_Flag                                 0
C2_Workplace closing                    0
C2_Flag                                 0
C3_Cancel public events                 0
C3_Flag                                 0
C4_Restrictions on gatherings           0
C4_Flag                                 0
C5_Close public transport               0
C5_Flag                                 0
C6_Stay at home requirements            0
C6_Flag                                 0
C7_Restrictions on internal movement    0
C7_Flag                                 0
C8_International travel controls        0
H1_Public information campaigns         0
H1_Flag                                 0
E1_Income support                       0
E1_Flag                                 0
dtype: int64

In [18]:
df_CD.columns

Index(['date', 'iso_code', 'population', 'population_density', 'median_age',
       'total_cases', 'total_deaths', 'new_cases', 'new_deaths'],
      dtype='object')

In [19]:
# Rename the Date and CountryCode columns in the Government Response dataframe to date and iso_code, respectively,
# to avoid using left_on and right_on in the merge operation

df_GR = df_GR.rename(columns={"Date":"date","CountryCode":"iso_code"})

In [20]:
# This function converts a date string of the format YYYYMMDD to YYYY-MM-DD

def formatDate(dt):
    d_str = str(dt)
    year = d_str[0:4]
    month = d_str[4:6]
    day = d_str[6:8]
    dt_str = year + "-" + month + "-" + day
     
    return dt_str

In [21]:
# Format the date column of the Government Response

df_GR["date"] = df_GR["date"].apply(lambda x:formatDate(x))

In [22]:
df_GR.head()

,date,iso_code,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,C5_Flag,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,H1_Public information campaigns,H1_Flag,E1_Income support,E1_Flag
0,2020-01-01,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-02,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-03,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-04,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-05,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df_GR.shape

(45140, 21)

In [24]:
df_CD.head()

,date,iso_code,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,106766.0,584.8,41.2,2.0,0.0,2.0,0.0
1,2020-03-20,ABW,106766.0,584.8,41.2,4.0,0.0,2.0,0.0
2,2020-03-24,ABW,106766.0,584.8,41.2,12.0,0.0,8.0,0.0
3,2020-03-25,ABW,106766.0,584.8,41.2,17.0,0.0,5.0,0.0
4,2020-03-26,ABW,106766.0,584.8,41.2,19.0,0.0,2.0,0.0


In [25]:
# Merge the Government Response and the COVID Data dataframe on date and iso_code

final_df = df_GR.merge(df_CD,on=["date","iso_code"])
final_df

,date,iso_code,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,C4_Flag,...,H1_Flag,E1_Income support,E1_Flag,population,population_density,median_age,total_cases,total_deaths,new_cases,new_deaths
0,2020-03-13,ABW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,106766.0,584.800,41.2,2.0,0.0,2.0,0.0
1,2020-03-20,ABW,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106766.0,584.800,41.2,4.0,0.0,2.0,0.0
2,2020-03-24,ABW,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106766.0,584.800,41.2,12.0,0.0,8.0,0.0
3,2020-03-25,ABW,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106766.0,584.800,41.2,17.0,0.0,5.0,0.0
4,2020-03-26,ABW,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,106766.0,584.800,41.2,19.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31800,2020-08-27,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6251.0,179.0,55.0,13.0
31801,2020-08-28,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6292.0,189.0,41.0,10.0
31802,2020-08-29,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6388.0,195.0,96.0,6.0
31803,2020-08-30,ZWE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14862927.0,42.729,19.6,6406.0,196.0,18.0,1.0


In [26]:
# Create four dataframes, one for each feature set to be explored for predicting future total cases and total deaths

df1 = final_df[['date','iso_code','population','population_density','median_age','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls','total_cases','total_deaths']]
df2 = final_df[['date','iso_code','population','population_density','median_age','C1_School closing','C2_Workplace closing','C3_Cancel public events','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls','total_cases','total_deaths']]
df3 = final_df[['date','iso_code','population','population_density','median_age','C1_School closing','C2_Workplace closing','C3_Cancel public events','C4_Restrictions on gatherings','C5_Close public transport','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls','total_cases','total_deaths']]
df4 = final_df[['date','iso_code','population','population_density','median_age','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls','E1_Income support','total_cases','total_deaths']]

# Put the dataframe in a list
dfs = [df1, df2, df3, df4]


In [27]:
# The following function creates a dataframe for each individual iso_code (country code), which contains the 
# following information:
# - date, iso_code, future date

# It returns a dictionary of the country codes dataframes and a list of the iso_codes used as dictionary key values

# Input: 
# df - A list of dataframes

# Returns:
# - A countries dictionary, each element of which contains future date dictionaries, each of which has 
# three columns: data, iso_code, and future date
# - A iso_codes list contain a unique list of all the country codes

def get_iso_dicts(df):
    
    # Get a lost of the iso_codes
    iso_codes = df["iso_code"].unique()
    
    # Initialize the country codes dictionary
    iso_dicts = {}
    
    # Iterate through the iso_cides
    for code in iso_codes:
        
        # Create a dictionary for the current iso_code
        iso_dicts[code] = {}
        
        # For each value of day in the future list (30,45,60, and 75)
        for day in future:
            
            # Create the future date column name
            date_col = "_".join(["date",day])
            
            # Copy the date and iso_code columns to a new dataframe, df_iso
            df_iso = final_df[final_df["iso_code"]==code][["date","iso_code"]]
            
            # Set the future date column to the current date + the current value of day
            df_iso[date_col] = pd.to_datetime(df_iso["date"]) + timedelta(days=int(day))
            
            # Set the type of the future date column to str so that it can be used in merge operations
            df_iso[date_col] = df_iso[date_col].astype(str)

            # Add the dataframe to the iso_dicts country dictionaries 
            iso_dicts[code][day] = df_iso
            
    # Return the country dataframes and the country codes        
    return iso_dicts, iso_codes

In [28]:
# This function groups all the country-day dataframe by the number of days into the future (30,45,60,75) into
# a list of four dataframes
#
# Input:
# iso_codes - country codes list
# iso_dicts - country-future-dates dictionaries
#
# Returns:
# A list of dataframes, one for each of the future days being considered
def get_cum_days(*iso_codes, **iso_dicts):
    
    # Initialize the days dataframe list
    days = []

    # For each day value in futures ('30','45','60','75')
    for day in future:
        
        # Create a new cumulative dataframe
        cum_df = pd.DataFrame()
        
        # For each iso code (country code)
        for code in iso_codes:
            
            # Append the dataframe corresponding to the country and the future days to the current cumulative dataframe
            cum_df = cum_df.append(iso_dicts[code][day])

        # Add the current cumulative dataframe to the days dataframe list
        days.append(cum_df.reset_index(drop=True))
    
    return days

In [29]:
# This function creates the future total cases and future total deaths columns by left joining the days dataframes
# with the original dataframe on the future date, the current date, and the iso_code columns.

# Input:
# days - list of future date dataframe

# Return:
# m_days - list of dataframes containing future total deaths and future total cases

def get_amended_days(*days):
        
    # Initialize the amended days dataframe list
    m_days = []
    
    # Iterate through the values in the future list ['30','45','60','75']
    for i,day in enumerate(future):
        
        # Merge the current days dataframe with the original dataframe and append it to m_days
        m_days.append(days[i].merge(final_df,left_on=["date_" + day,"iso_code"],right_on=["date","iso_code"],how="left",suffixes=["","_" + day]))
        
        # Drop the index of the merged dataframe
        m_days[i] = m_days[i].reset_index(drop=True)
        
        # Choose the useful columns from the merged dataframe
        m_days[i] = m_days[i][["date","iso_code","date_" + day,"total_cases","total_deaths"]]
        
        # Rename the total_cases and total_deaths case to identify to which future period they belong
        m_days[i].rename(columns={"total_cases":"total_cases_" + day,
                           "total_deaths":"total_deaths_" + day},inplace=True)
        
    return m_days

In [30]:
# This function creates the list of day dataframes, one for each of the future periods under consideration. 
# The dataframes contain the following columns: date, iso_code, future_date, future total cases, future total deaths 

# Initialize the future days list 
future = ["30","45","60","75"]

def get_days(df):
    
    # Get the country-days dicts and the country iso codes
    iso_dicts, iso_codes = get_iso_dicts(df)
    
    # Get the list of days dataframes for each of the future period lengths
    days = get_cum_days(*iso_codes, **iso_dicts)
        
    # Append to the list the future total deaths and the future total cases
    days = get_amended_days(*days)

    return days

In [31]:
# Earlier EDA has shown that, out of the researched neural network configurations, the following neural 
# network configuration produces the most accurate results

layers = { 
            "number_input_features": 0,   # number of input features will vary, value set elsewhere
            "n_layers": 4, 
            "l0":{"number_hidden_nodes":18,"activation_function":"relu"},
            "l1":{"number_hidden_nodes":9,"activation_function":"relu"},
            "l2":{"number_hidden_nodes":4,"activation_function":"relu"},
            "l3":{"number_hidden_nodes":1,"activation_function":"linear"}
        }

In [32]:
# Regression Analysis

# Initialize the list of models
models = []

# For each of the feature lists under consideration
for n,df in enumerate(dfs):

    # Get the features
    features = df.columns
    
    # Get the list of future days dataframes
    days = get_days(df)
        
    # For each of the targets, namely total_cases and total_deaths
    for target in ["total_cases","total_deaths"]:
        
        print(f"{target} %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%>\n\n")
        
        # For each day in future ['30','45','60','75']
        for i,day in enumerate(future):
            
            print(f"\n\n>>>>>>>> {day} >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ")
            
            # Merge the feature dataframe with the day dataframe
            df_n = df.merge(days[i],on=["date","iso_code"],how="left")
            print(df_n.columns)
            
            # Remove the target columns from merged dataframe
            features = [f for f in features if re.search("_\d{2}$",f) == None]
            features = features[2:]  # Remove date and iso_code
            
            # Set the number of input features for the neural networks based on the current feature set
            layers["number_input_features"] = len(features)

            # Create an instance of the ml class to start machine learning
            md = ml(df=df_n,
                    feature_set=n,
                    test_size=0.05,
                    title=f"{target} - {day} days out",
                    target=target,
                    period=day,
                    **layers
                   )
            
            # Train and test the machine learning mode over 100 epochs
            md.train_test(epochs=100)

            # Append the instance of the ml class into the models list
            models.append(md)


total_cases %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%>




>>>>>>>> 30 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> 
Index(['date', 'iso_code', 'population', 'population_density', 'median_age',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'total_cases', 'total_deaths',
       'date_30', 'date_30', 'total_cases_30', 'total_deaths_30'],
      dtype='object')


 Training and testing - 30 days ahead


target_n = total_cases_30

features = ['population', 'population_density', 'median_age', 'C6_Stay at home requirements', 'C7_Restrictions on internal movement', 'C8_International travel controls', 'total_cases', 'total_deaths']
population

population_density

median_age

C6_Stay at home requirements

C7_Restrictions on internal movement

C8_International travel controls

total_cases

total_deaths

Model: "sequential"
_________________________________________________________________
Layer (

793/793 [==============================] - 1s 2ms/step - loss: 0.0028 - mse: 0.0028
Epoch 68/100
793/793 [==============================] - 1s 2ms/step - loss: 0.0029 - mse: 0.0029
Epoch 69/100
793/793 [==============================] - 1s 2ms/step - loss: 0.0026 - mse: 0.0026
Epoch 70/100
793/793 [==============================] - 1s 2ms/step - loss: 0.0029 - mse: 0.0029
Epoch 71/100
793/793 [==============================] - 1s 2ms/step - loss: 0.0028 - mse: 0.0028
Epoch 72/100
793/793 [==============================] - 1s 2ms/step - loss: 0.0028 - mse: 0.0028
Epoch 73/100
793/793 [==============================] - 1s 2ms/step - loss: 0.0025 - mse: 0.0025
Epoch 74/100
793/793 [==============================] - 2s 2ms/step - loss: 0.0028 - mse: 0.0028
Epoch 75/100
793/793 [==============================] - 1s 2ms/step - loss: 0.0032 - mse: 0.0032
Epoch 76/100
793/793 [==============================] - 1s 2ms/step - loss: 0.0030 - mse: 0.0030
Epoch 77/100
793/793 [=====================

721/721 [==============================] - 1s 2ms/step - loss: 0.0067 - mse: 0.0067
Epoch 34/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0070 - mse: 0.0070
Epoch 35/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0064 - mse: 0.0064
Epoch 36/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0073 - mse: 0.0073
Epoch 37/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0060 - mse: 0.0060
Epoch 38/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0068 - mse: 0.0068
Epoch 39/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0059 - mse: 0.0059
Epoch 40/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0062 - mse: 0.0062
Epoch 41/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0058 - mse: 0.0058
Epoch 42/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0059 - mse: 0.0059
Epoch 43/100
721/721 [=====================

648/648 [==============================] - 1s 2ms/step - loss: 0.4526 - mse: 0.4526
Epoch 2/100
648/648 [==============================] - 1s 2ms/step - loss: 0.1028 - mse: 0.1028
Epoch 3/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0785 - mse: 0.0785
Epoch 4/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0609 - mse: 0.0609
Epoch 5/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0487 - mse: 0.0487
Epoch 6/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0389 - mse: 0.0389
Epoch 7/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0327 - mse: 0.0327
Epoch 8/100
648/648 [==============================] - 2s 3ms/step - loss: 0.0288 - mse: 0.0288
Epoch 9/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0251 - mse: 0.0251
Epoch 10/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0226 - mse: 0.0226
Epoch 11/100
648/648 [=============================

648/648 [==============================] - 1s 1ms/step - loss: 0.0062 - mse: 0.0062
Epoch 86/100
648/648 [==============================] - 1s 1ms/step - loss: 0.0063 - mse: 0.0063
Epoch 87/100
648/648 [==============================] - 1s 1ms/step - loss: 0.0073 - mse: 0.0073
Epoch 88/100
648/648 [==============================] - 2s 2ms/step - loss: 0.0068 - mse: 0.0068
Epoch 89/100
648/648 [==============================] - 4s 5ms/step - loss: 0.0063 - mse: 0.0063
Epoch 90/100
648/648 [==============================] - 3s 5ms/step - loss: 0.0062 - mse: 0.0062
Epoch 91/100
648/648 [==============================] - 3s 4ms/step - loss: 0.0063 - mse: 0.0063
Epoch 92/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0062 - mse: 0.0062
Epoch 93/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0059 - mse: 0.0059
Epoch 94/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0060 - mse: 0.0060
Epoch 95/100
648/648 [=====================

...

721/721 [==============================] - 1s 2ms/step - loss: 0.2660 - mse: 0.2660
Epoch 2/100
721/721 [==============================] - 1s 2ms/step - loss: 0.1080 - mse: 0.1080
Epoch 3/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0897 - mse: 0.0897
Epoch 4/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0758 - mse: 0.0758
Epoch 5/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0651 - mse: 0.0651
Epoch 6/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0564 - mse: 0.0564
Epoch 7/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0509 - mse: 0.0509
Epoch 8/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0467 - mse: 0.0467
Epoch 9/100
721/721 [==============================] - ETA: 0s - loss: 0.0434 - mse: 0.043 - 1s 2ms/step - loss: 0.0437 - mse: 0.0437
Epoch 10/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0416 - mse: 0.0416
Epoch 11/100


721/721 [==============================] - 1s 2ms/step - loss: 0.0105 - mse: 0.0105
Epoch 86/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0113 - mse: 0.0113
Epoch 87/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0102 - mse: 0.0102
Epoch 88/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0099 - mse: 0.0099
Epoch 89/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0099 - mse: 0.0099
Epoch 90/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0102 - mse: 0.0102
Epoch 91/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0095 - mse: 0.0095
Epoch 92/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0102 - mse: 0.0102
Epoch 93/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0100 - mse: 0.0100
Epoch 94/100
721/721 [==============================] - 1s 2ms/step - loss: 0.0098 - mse: 0.0098
Epoch 95/100
721/721 [=====================

648/648 [==============================] - 1s 2ms/step - loss: 0.0231 - mse: 0.0231
Epoch 51/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0215 - mse: 0.0215
Epoch 52/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0202 - mse: 0.0202
Epoch 53/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0217 - mse: 0.0217
Epoch 54/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0217 - mse: 0.0217
Epoch 55/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0201 - mse: 0.0201
Epoch 56/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0207 - mse: 0.0207
Epoch 57/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0219 - mse: 0.0219
Epoch 58/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0194 - mse: 0.0194
Epoch 59/100
648/648 [==============================] - 1s 2ms/step - loss: 0.0197 - mse: 0.0197
Epoch 60/100
648/648 [=====================

582/582 [==============================] - 1s 2ms/step - loss: 0.0569 - mse: 0.0569
Epoch 16/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0534 - mse: 0.0534
Epoch 17/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0505 - mse: 0.0505
Epoch 18/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0488 - mse: 0.0488
Epoch 19/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0450 - mse: 0.0450
Epoch 20/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0437 - mse: 0.0437
Epoch 21/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0425 - mse: 0.0425
Epoch 22/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0398 - mse: 0.0398
Epoch 23/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0379 - mse: 0.0379
Epoch 24/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0379 - mse: 0.0379
Epoch 25/100
582/582 [=====================

582/582 [==============================] - 1s 2ms/step - loss: 0.0253 - mse: 0.0253
Epoch 100/100
582/582 [==============================] - 1s 2ms/step - loss: 0.0256 - mse: 0.0256
Training r2_score = 0.972705141006838
Testing  r2_score = 0.9338160302527773


In [33]:
for md in models:
    df = md.get_df()
    print(df.columns)

Index(['date', 'iso_code', 'population', 'population_density', 'median_age',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'total_cases', 'total_deaths',
       'total_cases_30'],
      dtype='object')
Index(['date', 'iso_code', 'population', 'population_density', 'median_age',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'total_cases', 'total_deaths',
       'total_cases_45'],
      dtype='object')
Index(['date', 'iso_code', 'population', 'population_density', 'median_age',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8_International travel controls', 'total_cases', 'total_deaths',
       'total_cases_60'],
      dtype='object')
Index(['date', 'iso_code', 'population', 'population_density', 'median_age',
       'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
       'C8

In [ ]:
for md in models:
    df = md.get_df()
    model = md.get_model()
    features = md.get_features()
    X_scaler = md.get_X_scaler()
    y_scaler = md.get_y_scaler()
    
    target = md.get_target() + "_" + md.get_period()
    
    iso_codes = df["iso_code"].unique()
    
    for code in iso_codes:
        X = df[df["iso_code"] == code][features]
        
        X_scaled = X_scaler.transform(X)
        y_pred_scaled = model.predict(X_scaled)
        y_pred = y_scaler.inverse_transform(y_pred_scaled)
        
        df_ic = df[df["iso_code"] == code]
        
        time = [str(t) for t in df_ic["date"]]

        x_vals = df_ic["date"].values.tolist()
        y_test = df_ic[target].values.tolist()  
        y_pred_scaled = model.predict(X_scaled).reshape(-1,1).tolist()
        y_pred = [y_scaler.inverse_transform(y) for y in y_pred_scaled]
       
        
        df_c = pd.DataFrame({"Time":x_vals,"Actual":y_test, "Predicted": y_pred})

        plt.figure(figsize=[25,15])
        plt.plot(df_c["Time"], df_c["Actual"], c="Red")
        plt.plot(df_c["Time"], df_c["Predicted"], c="Green")
        
        plt.xlabel("Time")
        plt.ylabel("Actual/Predicted")
        
        plt.title(md.get_title() + "; Country: " + code)
        plt.xticks(rotation=90)
        plt.legend(["Actual","Predicted"])
        
        feature_str = ""
        for feature in features:
            if re.search('^[CH\d]',feature) == None:
                feature_str = feature_str + "_" + feature
            else:
                feature_str = feature_str + "_" + feature[:2]
                
        
        plt.savefig(f"Resources/graphs/target-{md.get_target()}-features-{feature_str}-period-{md.get_period()}-country-{code}.png")
        plt.close()
